<a href="https://colab.research.google.com/github/mitosagi/puzzdra-nnsolver/blob/master/baseline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 15puzzleソルバ
人間にとって直感的な解法を使用して15パズルを解きます。

In [29]:
import numpy as np

board_x = 5
board_y = 6

# Creating a 4x4 array with numbers 1 to 16 in order
ordered_puzzle = np.arange(1, board_y*board_x+1).reshape(board_y, board_x)

# Creating a 4x4 array with numbers 1 to 16 in random order
random_puzzle = np.random.permutation(board_y*board_x) + 1
random_puzzle = random_puzzle.reshape(board_y, board_x)

ordered_puzzle, random_puzzle

(array([[ 1,  2,  3,  4,  5],
        [ 6,  7,  8,  9, 10],
        [11, 12, 13, 14, 15],
        [16, 17, 18, 19, 20],
        [21, 22, 23, 24, 25],
        [26, 27, 28, 29, 30]]),
 array([[21, 18, 20,  3, 30],
        [24, 11,  5, 12, 26],
        [ 6, 10,  8, 14,  2],
        [22,  9, 15, 28,  4],
        [17,  1, 25, 19, 23],
        [29,  7, 27, 13, 16]]))

In [22]:
from queue import Queue

# Correcting the BFS function for a 5x6 board
def bfs_with_obstacle_matrix_5x6(start, obstacle_matrix):
    """ Perform BFS to calculate distances from a starting point on a 5x6 board, considering an obstacle matrix. """
    size_y, size_x = obstacle_matrix.shape  # Correcting the dimensions
    distances = np.full((size_y, size_x), 99)  # Initialize with 99
    q = Queue()

    # Start from the given position
    q.put(start)
    distances[start] = 0

    while not q.empty():
        current = q.get()
        current_distance = distances[current]

        # Check all four directions
        for dx, dy in [(-1, 0), (1, 0), (0, -1), (0, 1)]:
            next_pos = (current[0] + dy, current[1] + dx)

            if 0 <= next_pos[1] < size_x and 0 <= next_pos[0] < size_y and distances[next_pos] == 99 and obstacle_matrix[next_pos] == 0:
                distances[next_pos] = current_distance + 1
                q.put(next_pos)

    return distances



# Define the obstacle matrix for a 5x6 board (0 = no obstacle, 1 = obstacle)
obstacle_matrix_5x6 = np.zeros((6, 5), dtype=int)
obstacle_matrix_5x6[1, 0] = 1  # Set the obstacle at position (2, 1)
obstacle_matrix_5x6[2, 1] = 1  # Set the obstacle at position (2, 1)
obstacle_matrix_5x6[3, 1] = 1  # Set the obstacle at position (2, 1)

print([bfs_with_obstacle_matrix_5x6((i // 5, i % 5), obstacle_matrix_5x6) for i in [0]][0])

[[ 0  1  2  3  4]
 [99  2  3  4  5]
 [10 99  4  5  6]
 [ 9 99  5  6  7]
 [ 8  7  6  7  8]
 [ 9  8  7  8  9]]


In [31]:
import copy
print(random_puzzle)
step_n = 2
step_pos = tuple(np.argwhere(ordered_puzzle == step_n)[0])
print(step_pos)
tile_n = 2
tile_pos = tuple(np.argwhere(random_puzzle == tile_n)[0])
print(tile_pos)

tile_cost = bfs_with_obstacle_matrix_5x6(step_pos, obstacle_matrix_5x6)
print(tile_cost)

print(tile_cost[tile_pos])
for i in range(tile_cost[tile_pos]):
  move_obstacle = copy.deepcopy(obstacle_matrix_5x6)
  move_obstacle[tile_pos] = 99

  move_cost = bfs_with_obstacle_matrix_5x6(move_pos, move_obstacle)
  print(move_cost)

  move_pos = tuple(np.argwhere(random_puzzle == board_y*board_x)[0])




[[21 18 20  3 30]
 [24 11  5 12 26]
 [ 6 10  8 14  2]
 [22  9 15 28  4]
 [17  1 25 19 23]
 [29  7 27 13 16]]
(0, 1)
(2, 4)
[[ 1  0  1  2  3]
 [99  1  2  3  4]
 [ 9 99  3  4  5]
 [ 8 99  4  5  6]
 [ 7  6  5  6  7]
 [ 8  7  6  7  8]]
5
[[ 4  3  2  1  0]
 [99  4  3  2  1]
 [10 99  4  3 99]
 [ 9 99  5  4  5]
 [ 8  7  6  5  6]
 [ 9  8  7  6  7]]
[[ 4  3  2  1  0]
 [99  4  3  2  1]
 [10 99  4  3 99]
 [ 9 99  5  4  5]
 [ 8  7  6  5  6]
 [ 9  8  7  6  7]]
[[ 4  3  2  1  0]
 [99  4  3  2  1]
 [10 99  4  3 99]
 [ 9 99  5  4  5]
 [ 8  7  6  5  6]
 [ 9  8  7  6  7]]
[[ 4  3  2  1  0]
 [99  4  3  2  1]
 [10 99  4  3 99]
 [ 9 99  5  4  5]
 [ 8  7  6  5  6]
 [ 9  8  7  6  7]]
[[ 4  3  2  1  0]
 [99  4  3  2  1]
 [10 99  4  3 99]
 [ 9 99  5  4  5]
 [ 8  7  6  5  6]
 [ 9  8  7  6  7]]
